In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import torch
import numpy as np
import _init_paths

from models.losses import FocalLoss
from models.losses import RegL1Loss, RegLoss, NormRegL1Loss, RegWeightedL1Loss
from models.decode import ctdet_decode
from models.utils import _sigmoid
from utils.debugger import Debugger
from utils.post_process import ctdet_post_process
from utils.oracle_utils import gen_oracle_map
from trains.base_trainer import BaseTrainer


In [ ]:
class CtdetLoss(torch.nn.Module):
  def __init__(self, opt):
    super(CtdetLoss, self).__init__()
    self.crit = torch.nn.MSELoss() if opt.mse_loss else FocalLoss()
    self.crit_reg = RegL1Loss() if opt.reg_loss == 'l1' else \
              RegLoss() if opt.reg_loss == 'sl1' else None
    self.crit_wh = torch.nn.L1Loss(reduction='sum') if opt.dense_wh else \
              NormRegL1Loss() if opt.norm_wh else \
              RegWeightedL1Loss() if opt.cat_spec_wh else self.crit_reg
    self.opt = opt

  def forward(self, outputs, batch):
    opt = self.opt
    hm_loss, wh_loss, off_loss = 0, 0, 0
    for s in range(opt.num_stacks):
      output = outputs[s]
      if not opt.mse_loss:
        output['hm'] = _sigmoid(output['hm'])

      if opt.eval_oracle_hm:
        output['hm'] = batch['hm']
      if opt.eval_oracle_wh:
        output['wh'] = torch.from_numpy(gen_oracle_map(
          batch['wh'].detach().cpu().numpy(), 
          batch['ind'].detach().cpu().numpy(), 
          output['wh'].shape[3], output['wh'].shape[2])).to(opt.device)
      if opt.eval_oracle_offset:
        output['reg'] = torch.from_numpy(gen_oracle_map(
          batch['reg'].detach().cpu().numpy(), 
          batch['ind'].detach().cpu().numpy(), 
          output['reg'].shape[3], output['reg'].shape[2])).to(opt.device)

      hm_loss += self.crit(output['hm'], batch['hm']) / opt.num_stacks
      if opt.wh_weight > 0:
        if opt.dense_wh:
          mask_weight = batch['dense_wh_mask'].sum() + 1e-4
          wh_loss += (
            self.crit_wh(output['wh'] * batch['dense_wh_mask'],
            batch['dense_wh'] * batch['dense_wh_mask']) / 
            mask_weight) / opt.num_stacks
        elif opt.cat_spec_wh:
          wh_loss += self.crit_wh(
            output['wh'], batch['cat_spec_mask'],
            batch['ind'], batch['cat_spec_wh']) / opt.num_stacks
        else:
          wh_loss += self.crit_reg(
            output['wh'], batch['reg_mask'],
            batch['ind'], batch['wh']) / opt.num_stacks
      
      if opt.reg_offset and opt.off_weight > 0:
        off_loss += self.crit_reg(output['reg'], batch['reg_mask'],
                             batch['ind'], batch['reg']) / opt.num_stacks
        
    loss = opt.hm_weight * hm_loss + opt.wh_weight * wh_loss + \
           opt.off_weight * off_loss
    loss_stats = {'loss': loss, 'hm_loss': hm_loss,
                  'wh_loss': wh_loss, 'off_loss': off_loss}
    return loss, loss_stats

In [ ]:
class CtdetTrainer(BaseTrainer):
  def __init__(self, opt, model, optimizer=None):
    super(CtdetTrainer, self).__init__(opt, model, optimizer=optimizer)
  
  def _get_losses(self, opt):
    loss_states = ['loss', 'hm_loss', 'wh_loss', 'off_loss']
    loss = CtdetLoss(opt)
    return loss_states, loss

  def debug(self, batch, output, iter_id):
    opt = self.opt
    reg = output['reg'] if opt.reg_offset else None
    dets = ctdet_decode(
      output['hm'], output['wh'], reg=reg,
      cat_spec_wh=opt.cat_spec_wh, K=opt.K)
    dets = dets.detach().cpu().numpy().reshape(1, -1, dets.shape[2])
    dets[:, :, :4] *= opt.down_ratio
    dets_gt = batch['meta']['gt_det'].numpy().reshape(1, -1, dets.shape[2])
    dets_gt[:, :, :4] *= opt.down_ratio
    for i in range(1):
      debugger = Debugger(
        dataset=opt.dataset, ipynb=(opt.debug==3), theme=opt.debugger_theme)
      img = batch['input'][i].detach().cpu().numpy().transpose(1, 2, 0)
      img = np.clip(((
        img * opt.std + opt.mean) * 255.), 0, 255).astype(np.uint8)
      pred = debugger.gen_colormap(output['hm'][i].detach().cpu().numpy())
      gt = debugger.gen_colormap(batch['hm'][i].detach().cpu().numpy())
      debugger.add_blend_img(img, pred, 'pred_hm')
      debugger.add_blend_img(img, gt, 'gt_hm')
      debugger.add_img(img, img_id='out_pred')
      for k in range(len(dets[i])):
        if dets[i, k, 4] > opt.center_thresh:
          debugger.add_coco_bbox(dets[i, k, :4], dets[i, k, -1],
                                 dets[i, k, 4], img_id='out_pred')

      debugger.add_img(img, img_id='out_gt')
      for k in range(len(dets_gt[i])):
        if dets_gt[i, k, 4] > opt.center_thresh:
          debugger.add_coco_bbox(dets_gt[i, k, :4], dets_gt[i, k, -1],
                                 dets_gt[i, k, 4], img_id='out_gt')

      if opt.debug == 4:
        debugger.save_all_imgs(opt.debug_dir, prefix='{}'.format(iter_id))
      else:
        debugger.show_all_imgs(pause=True)

In [ ]:
def save_result(self, output, batch, results):
    reg = output['reg'] if self.opt.reg_offset else None
    dets = ctdet_decode(
      output['hm'], output['wh'], reg=reg,
      cat_spec_wh=self.opt.cat_spec_wh, K=self.opt.K)
    dets = dets.detach().cpu().numpy().reshape(1, -1, dets.shape[2])
    dets_out = ctdet_post_process(
      dets.copy(), batch['meta']['c'].cpu().numpy(),
      batch['meta']['s'].cpu().numpy(),
      output['hm'].shape[2], output['hm'].shape[3], output['hm'].shape[1])
    results[batch['meta']['img_id'].cpu().numpy()[0]] = dets_out[0]

In [ ]:
import torch

In [19]:
output = torch.arange(0,250).view(1,10,5,5)
output.shape
output.

torch.Size([1, 10, 5, 5])

In [13]:
ind = torch.arange(0,20).view(1,4,5)
ind.shape

torch.Size([1, 4, 5])

In [20]:
output = output.permute(0,2,3,1)
output.shape

torch.Size([1, 5, 5, 10])

In [22]:
output = output.view(1, -1, 10)
output.shape

torch.Size([1, 25, 10])

In [16]:
ind

tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]]])

In [23]:
ind1 = ind.unsqueeze(3)
ind1.shape

torch.Size([1, 4, 5, 1])

In [29]:
ind1 = ind1.expand(ind1.size(0),ind1.size(1),ind1.size(2),output.size(2)//ind1.size(2))
ind1.shape

torch.Size([1, 4, 5, 2])

In [28]:
output.size(2)//ind1.size(2)

2

In [52]:
org = torch.arange(0,20).view(1,4,5)
org[0]

tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])

In [41]:
ind1.shape

torch.Size([1, 4, 5, 2])

In [49]:
ind2 = ind1.clone()
ind2 = ind2.view(1,4,-1)
ind2

tensor([[[ 0,  0,  1,  1,  2,  2,  3,  3,  4,  4],
         [ 5,  5,  6,  6,  7,  7,  8,  8,  9,  9],
         [10, 10, 11, 11, 12, 12, 13, 13, 14, 14],
         [15, 15, 16, 16, 17, 17, 18, 18, 19, 19]]])

In [59]:
new_out = output.gather(1, ind2)
new_out.shape
new_out

tensor([[[  0,  25,  51,  76, 102, 127, 153, 178, 204, 229],
         [  5,  30,  56,  81, 107, 132, 158, 183, 209, 234],
         [ 10,  35,  61,  86, 112, 137, 163, 188, 214, 239],
         [ 15,  40,  66,  91, 117, 142, 168, 193, 219, 244]]])

In [46]:
ind3 = ind1.clone()
ind3 = ind3.permute(0,1,3,2)
ind3.shape

torch.Size([1, 4, 2, 5])

In [50]:
ind4 = ind3.clone()
ind4 = ind4.view(1,4,-1)
ind4

tensor([[[ 0,  1,  2,  3,  4,  0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9,  5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14, 10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19, 15, 16, 17, 18, 19]]])

In [56]:
img = torch.arange(0,16).view(1,4,4)
img

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15]]])

In [57]:
new = img.clone()
new = new.view(1,-1)
new

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]])

In [64]:

def _gather_feat(feat, ind, mask=None):
    dim  = feat.size(2)
    ind  = ind.unsqueeze(3).expand(ind.size(0), ind.size(1),ind.size(2), dim//ind.size(2)).contiguous().view(ind.size(0), ind.size(1), -1)
    feat = feat.gather(1, ind)
    if mask is not None:
        mask = mask.unsqueeze(2).expand_as(feat)
        feat = feat[mask]
        feat = feat.view(-1, dim)
    return feat

In [67]:
outp = torch.arange(0,250).view(1, 10, 5,5)
outp = outp.permute(0,2,3,1)
feat = outp.view(1,-1,10)
ind = torch.arange(0,20).view(1,4,5)
_gather_feat(feat, ind, mask=None).shape

torch.Size([1, 4, 10])

In [69]:
mask = torch.ones((1, 4, 5))
mask

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]])

In [75]:
mask1 = mask.unsqueeze(3).expand(1,4,5,2).contiguous().view(1,4,-1)
mask1 = mask.unsqueeze(3).expand(mask.size(0),mask.size(1),mask.size(2),2).contiguous().view(mask.size(0),mask.size(1),-1).float()
mask1.shape

torch.Size([1, 4, 10])

In [89]:
from torch.autograd import Variable

x = Variable(3*torch.ones(2, 2), requires_grad = True)

y1 = x[0] + 2

y2 = x[1] *x[1]

z1 = y1 * y1 * 3
z2 = y2*y2*3

out = (z2+z1).mean()

# 现在我们来使用反向传播

out.backward()

# out.backward()和操作out.backward(torch.Tensor([1.0]))是等价的

# 在此处输出 d(out)/dx

x.grad


tensor([[ 15.,  15.],
        [162., 162.]])

In [90]:
z1

tensor([75., 75.], grad_fn=<MulBackward>)

In [91]:
z2

tensor([243., 243.], grad_fn=<MulBackward>)

In [94]:
pred = torch.ones((1, 4, 10))
print(pred.shape)
pred = pred[...,0:2]
print(pred.shape)

torch.Size([1, 4, 10])
torch.Size([1, 4, 2])


In [96]:
t = 0
for i in range(5):
    t += 2
print(t)

10


In [155]:
main_points_num=3
cls_num = 4
K = 10
height,width = 5,5
s = torch.arange(0,main_points_num*cls_num*25).view(1, main_points_num*cls_num, 5,5)
print(s)
s = s.contiguous().view(1, main_points_num*cls_num, 25)
print(s)

topk_scores, topk_inds = torch.topk(s.view(1, main_points_num*cls_num, -1), K)
print(topk_scores)
print(topk_inds)

tensor([[[[  0,   1,   2,   3,   4],
          [  5,   6,   7,   8,   9],
          [ 10,  11,  12,  13,  14],
          [ 15,  16,  17,  18,  19],
          [ 20,  21,  22,  23,  24]],

         [[ 25,  26,  27,  28,  29],
          [ 30,  31,  32,  33,  34],
          [ 35,  36,  37,  38,  39],
          [ 40,  41,  42,  43,  44],
          [ 45,  46,  47,  48,  49]],

         [[ 50,  51,  52,  53,  54],
          [ 55,  56,  57,  58,  59],
          [ 60,  61,  62,  63,  64],
          [ 65,  66,  67,  68,  69],
          [ 70,  71,  72,  73,  74]],

         [[ 75,  76,  77,  78,  79],
          [ 80,  81,  82,  83,  84],
          [ 85,  86,  87,  88,  89],
          [ 90,  91,  92,  93,  94],
          [ 95,  96,  97,  98,  99]],

         [[100, 101, 102, 103, 104],
          [105, 106, 107, 108, 109],
          [110, 111, 112, 113, 114],
          [115, 116, 117, 118, 119],
          [120, 121, 122, 123, 124]],

         [[125, 126, 127, 128, 129],
          [130, 131, 132, 13

In [135]:
topk_scoress = topk_scores.contiguous().view(1,cls_num, main_points_num, -1)
print(topk_scoress)
topk_scoress = topk_scoress.permute(0,2,1,3).contiguous().view(1,main_points_num,-1)
print(topk_scoress)

topk_indss = topk_inds.contiguous().view(1,cls_num, main_points_num, -1)
print(topk_indss)
topk_indss = topk_indss.permute(0,2,1,3).contiguous().view(1,main_points_num,-1)
print(topk_indss.shape)

tensor([[[[ 24,  23,  22,  21,  20,  19,  18,  17,  16,  15],
          [ 49,  48,  47,  46,  45,  44,  43,  42,  41,  40],
          [ 74,  73,  72,  71,  70,  69,  68,  67,  66,  65]],

         [[ 99,  98,  97,  96,  95,  94,  93,  92,  91,  90],
          [124, 123, 122, 121, 120, 119, 118, 117, 116, 115],
          [149, 148, 147, 146, 145, 144, 143, 142, 141, 140]],

         [[174, 173, 172, 171, 170, 169, 168, 167, 166, 165],
          [199, 198, 197, 196, 195, 194, 193, 192, 191, 190],
          [224, 223, 222, 221, 220, 219, 218, 217, 216, 215]],

         [[249, 248, 247, 246, 245, 244, 243, 242, 241, 240],
          [274, 273, 272, 271, 270, 269, 268, 267, 266, 265],
          [299, 298, 297, 296, 295, 294, 293, 292, 291, 290]]]])
tensor([[[ 24,  23,  22,  21,  20,  19,  18,  17,  16,  15,  99,  98,  97,  96,
           95,  94,  93,  92,  91,  90, 174, 173, 172, 171, 170, 169, 168, 167,
          166, 165, 249, 248, 247, 246, 245, 244, 243, 242, 241, 240],
         [ 49,  

In [139]:
topk_indss = topk_indss % (height * width)
topk_ys   = (topk_indss / width).int().float()
topk_xs   = (topk_indss % width).int().float()
print(topk_indss)
print(topk_indss.shape, topk_ys.shape, topk_xs.shape)

tensor([[[24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 24, 23, 22, 21, 20, 19, 18,
          17, 16, 15, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 24, 23, 22, 21,
          20, 19, 18, 17, 16, 15],
         [24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 24, 23, 22, 21, 20, 19, 18,
          17, 16, 15, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 24, 23, 22, 21,
          20, 19, 18, 17, 16, 15],
         [24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 24, 23, 22, 21, 20, 19, 18,
          17, 16, 15, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 24, 23, 22, 21,
          20, 19, 18, 17, 16, 15]]])
torch.Size([1, 3, 40]) torch.Size([1, 3, 40]) torch.Size([1, 3, 40])


In [141]:
topk_score, topk_ind = torch.topk(topk_scoress, K)
print(topk_score)
print(topk_ind)
print(topk_score.shape)
print(topk_ind.shape)

tensor([[[249, 248, 247, 246, 245, 244, 243, 242, 241, 240],
         [274, 273, 272, 271, 270, 269, 268, 267, 266, 265],
         [299, 298, 297, 296, 295, 294, 293, 292, 291, 290]]])
tensor([[[30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
         [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
         [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]]])
torch.Size([1, 3, 10])
torch.Size([1, 3, 10])


In [142]:
topk_clses = (topk_ind / K).int()
print(topk_clses)
print(topk_clses.shape)

tensor([[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
         [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
         [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]]], dtype=torch.int32)
torch.Size([1, 3, 10])


In [165]:
topk_indff = topk_indss.permute(0, 2,1)
topk_indf = topk_ind.permute(0,2,1)
topk_indsssf = topk_indff.gather(1, topk_indf)
print('topk_indsssf',topk_indsssf.shape)
topk_ysf =  topk_ys.permute(0, 2,1)
topk_xsf =  topk_xs.permute(0, 2,1)
topk_ysssf = topk_ysf.gather(1, topk_indf)
print('topk_ysssf',topk_ysssf)
topk_xsssf = topk_xsf.gather(1, topk_indf)
print('topk_xsssf',topk_xsssf)

topk_indsssf torch.Size([1, 10, 3])
topk_ysssf tensor([[[4., 4., 4.],
         [4., 4., 4.],
         [4., 4., 4.],
         [4., 4., 4.],
         [4., 4., 4.],
         [3., 3., 3.],
         [3., 3., 3.],
         [3., 3., 3.],
         [3., 3., 3.],
         [3., 3., 3.]]])
topk_xsssf tensor([[[4., 4., 4.],
         [3., 3., 3.],
         [2., 2., 2.],
         [1., 1., 1.],
         [0., 0., 0.],
         [4., 4., 4.],
         [3., 3., 3.],
         [2., 2., 2.],
         [1., 1., 1.],
         [0., 0., 0.]]])


RuntimeError: Expected tensor [1 x 3 x 10], src [1 x 3 x 40] and index [1 x 3 x 10] to have the same size apart from dimension 1 at /opt/conda/conda-bld/pytorch_1535491974311/work/aten/src/TH/generic/THTensorMath.cpp:620

In [160]:
def _topk_gridneighbor(scores, main_points_num, K=40):
    batch, allchannels, height, width = scores.size()
    cat = allchannels//main_points_num 
    
    topk_scores, topk_inds = torch.topk(s.view(1, allchannels, -1), K)
    print(topk_scores)
    print(topk_inds)
    
    topk_scoress = topk_scores.contiguous().view(1,cat, main_points_num, -1)
    print(topk_scoress)
    topk_scoress = topk_scoress.permute(0,2,1,3).contiguous().view(1,main_points_num,-1)
    print(topk_scoress)

    topk_indss = topk_inds.contiguous().view(1,cat, main_points_num, -1)
    print(topk_indss)
    topk_indss = topk_indss.permute(0,2,1,3).contiguous().view(1,main_points_num,-1)
    print(topk_indss.shape)
    
    topk_indss = topk_indss % (height * width)
    topk_ys   = (topk_indss / width).int().float()
    topk_xs   = (topk_indss % width).int().float()
    print(topk_indss)
    print(topk_indss.shape, topk_ys.shape, topk_xs.shape)
    
    topk_score, topk_ind = torch.topk(topk_scoress, K)
    print(topk_score)
    print(topk_ind)
    print(topk_score.shape)
    print(topk_ind.shape)
    
    topk_clses = (topk_ind / K).int()
    print(topk_clses)
    print(topk_clses.shape)
    
    topk_indff = topk_indss.permute(0, 2,1)
    topk_indf = topk_ind.permute(0,2,1)
    topk_indsssf = topk_indff.gather(1, topk_indf)
    print('topk_indsssf',topk_indsssf)
    topk_ysf =  topk_ys.permute(0, 2,1)
    topk_xsf =  topk_xs.permute(0, 2,1)
    topk_ysssf = topk_ysf.gather(1, topk_indf)
    print('topk_ysssf',topk_ysssf)
    topk_xsssf = topk_xsf.gather(1, topk_indf)
    print('topk_xsssf',topk_xsssf)

In [200]:
def _topk_gridneighbor(scores, main_points_num, K=40):
    batch, allchannels, height, width = scores.size()
    cat = allchannels//main_points_num 
    
    topk_scores, topk_inds = torch.topk(s.view(1, allchannels, -1), K)

    
    topk_scores = topk_scores.contiguous().view(1,cat, main_points_num, -1)

    topk_scores = topk_scores.permute(0,2,1,3).contiguous().view(1,main_points_num,-1)


    topk_inds = topk_inds.contiguous().view(1,cat, main_points_num, -1)

    topk_inds = topk_inds.permute(0,2,1,3).contiguous().view(1,main_points_num,-1)
    
    topk_inds = topk_inds % (height * width)
    topk_ys   = (topk_inds / width).int().float()
    topk_xs   = (topk_inds % width).int().float()
    
    topk_score, topk_ind = torch.topk(topk_scores, K)
    
    topk_clses = (topk_ind / K).int().permute(0,2,1)
    
    topk_inds = topk_inds.permute(0, 2,1)
    topk_ind = topk_ind.permute(0,2,1)
    topk_inds = topk_inds.gather(1, topk_ind)
    topk_ys =  topk_ys.permute(0, 2,1)
    topk_xs =  topk_xs.permute(0, 2,1)
    topk_ys = topk_ys.gather(1, topk_ind)
    topk_xs = topk_xs.gather(1, topk_ind)
    print('topk_ys',topk_ys)
    print('topk_xs',topk_xs)
    print('topk_score',topk_score.permute(0,2,1))
    print('topk_inds',topk_inds)
    print('topk_clses',topk_clses)

In [201]:
scores = torch.arange(0,300).view(1, 12, 5,5)
main_points_num = 3
_topk_gridneighbor(scores, main_points_num, K=10)

topk_ys tensor([[[4., 4., 4.],
         [4., 4., 4.],
         [4., 4., 4.],
         [4., 4., 4.],
         [4., 4., 4.],
         [3., 3., 3.],
         [3., 3., 3.],
         [3., 3., 3.],
         [3., 3., 3.],
         [3., 3., 3.]]])
topk_xs tensor([[[4., 4., 4.],
         [3., 3., 3.],
         [2., 2., 2.],
         [1., 1., 1.],
         [0., 0., 0.],
         [4., 4., 4.],
         [3., 3., 3.],
         [2., 2., 2.],
         [1., 1., 1.],
         [0., 0., 0.]]])
topk_score tensor([[[249, 274, 299],
         [248, 273, 298],
         [247, 272, 297],
         [246, 271, 296],
         [245, 270, 295],
         [244, 269, 294],
         [243, 268, 293],
         [242, 267, 292],
         [241, 266, 291],
         [240, 265, 290]]])
topk_inds tensor([[[24, 24, 24],
         [23, 23, 23],
         [22, 22, 22],
         [21, 21, 21],
         [20, 20, 20],
         [19, 19, 19],
         [18, 18, 18],
         [17, 17, 17],
         [16, 16, 16],
         [15, 15, 15]]])
topk_

In [202]:
reg = torch.arange(0,60).view(1, 10, 6)
ys = torch.arange(0,30).view(1, 10, 3)
print(reg)
print(ys)

tensor([[[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17],
         [18, 19, 20, 21, 22, 23],
         [24, 25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34, 35],
         [36, 37, 38, 39, 40, 41],
         [42, 43, 44, 45, 46, 47],
         [48, 49, 50, 51, 52, 53],
         [54, 55, 56, 57, 58, 59]]])
tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8],
         [ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17],
         [18, 19, 20],
         [21, 22, 23],
         [24, 25, 26],
         [27, 28, 29]]])


In [185]:
for i in range(main_points_num):
    ys[...,i] = ys[...,i] + reg[:,:, 2*i+1]
ys

tensor([[[ 1,  4,  7],
         [10, 13, 16],
         [19, 22, 25],
         [28, 31, 34],
         [37, 40, 43],
         [46, 49, 52],
         [55, 58, 61],
         [64, 67, 70],
         [73, 76, 79],
         [82, 85, 88]]])

In [182]:
for i in range(3):
    xs[...,i] = xs[...,i] + reg[:,:, 2*i]
xs

tensor([[[  0,   5,  10],
         [ 15,  20,  25],
         [ 30,  35,  40],
         [ 45,  50,  55],
         [ 60,  65,  70],
         [ 75,  80,  85],
         [ 90,  95, 100],
         [105, 110, 115],
         [120, 125, 130],
         [135, 140, 145]]])

In [223]:
a = xs[...,1].unsqueeze(2)
b = xs[...,2].unsqueeze(2)
wh = torch.cat([reg,reg,reg,reg],dim=2)
xs = torch.cat([xs,a],dim=2)
ys = torch.cat([ys,b],dim=2)
point_flags = [2,4,6,8]

In [222]:
c
r = []
r.append(c)
r.append(c)
torch.cat(r, dim=2)

tensor([[[  5,  10,   5,  10,   5,  10,   5,  10],
         [ 20,  25,  20,  25,  20,  25,  20,  25],
         [ 35,  40,  35,  40,  35,  40,  35,  40],
         [ 50,  55,  50,  55,  50,  55,  50,  55],
         [ 65,  70,  65,  70,  65,  70,  65,  70],
         [ 80,  85,  80,  85,  80,  85,  80,  85],
         [ 95, 100,  95, 100,  95, 100,  95, 100],
         [110, 115, 110, 115, 110, 115, 110, 115],
         [125, 130, 125, 130, 125, 130, 125, 130],
         [140, 145, 140, 145, 140, 145, 140, 145]]])

In [227]:
def bbox_decode(xs, ys, wh, point_flags):
    all_mainpoint_bboxes = []
    for idx, point in enumerate(point_flags):
        xs_i = xs[..., idx]
        ys_i = ys[..., idx]
        
        w_i = wh[..., 2*idx]
        h_i = wh[..., 2*idx+1]
        
        row = point // 3
        col = point % 3
        
        x1 = (xs_i - w_i).unsqueeze(2)
        y1 = (ys_i - h_i).unsqueeze(2)
        x2 = (xs_i + w_i*(3**(1-col))).unsqueeze(2)
        y2 = (ys_i + h_i*(3**(1-row))).unsqueeze(2)
        each_mainpoint_bboxes = torch.cat([x1, y1, x2, y2], dim=2)
        all_mainpoint_bboxes.append(each_mainpoint_bboxes)
    return torch.cat(all_mainpoint_bboxes, dim=2)

In [235]:
bbox_decode(xs, ys, wh, point_flags)[:,:,4].shape

torch.Size([1, 10])

5
4
3


In [1]:
color_list = np.array(
    [
        1.000, 1.000, 1.000,
        0.850, 0.325, 0.098,
        0.929, 0.694, 0.125,
        0.494, 0.184, 0.556,
        0.466, 0.674, 0.188,
        0.301, 0.745, 0.933,
        0.635, 0.078, 0.184,
        0.300, 0.300, 0.300,
        0.600, 0.600, 0.600,
        1.000, 0.000, 0.000,
        1.000, 0.500, 0.000,
        0.749, 0.749, 0.000,
        0.000, 1.000, 0.000,
        0.000, 0.000, 1.000,
        0.667, 0.000, 1.000,
        0.333, 0.333, 0.000,
        0.333, 0.667, 0.000,
        0.333, 1.000, 0.000,
        0.667, 0.333, 0.000,
        0.667, 0.667, 0.000,
        0.667, 1.000, 0.000,
        1.000, 0.333, 0.000,
        1.000, 0.667, 0.000,
        1.000, 1.000, 0.000,
        0.000, 0.333, 0.500,
        0.000, 0.667, 0.500,
        0.000, 1.000, 0.500,
        0.333, 0.000, 0.500,
        0.333, 0.333, 0.500,
        0.333, 0.667, 0.500,
        0.333, 1.000, 0.500,
        0.667, 0.000, 0.500,
        0.667, 0.333, 0.500,
        0.667, 0.667, 0.500,
        0.667, 1.000, 0.500,
        1.000, 0.000, 0.500,
        1.000, 0.333, 0.500,
        1.000, 0.667, 0.500,
        1.000, 1.000, 0.500,
        0.000, 0.333, 1.000,
        0.000, 0.667, 1.000,
        0.000, 1.000, 1.000,
        0.333, 0.000, 1.000,
        0.333, 0.333, 1.000,
        0.333, 0.667, 1.000,
        0.333, 1.000, 1.000,
        0.667, 0.000, 1.000,
        0.667, 0.333, 1.000,
        0.667, 0.667, 1.000,
        0.667, 1.000, 1.000,
        1.000, 0.000, 1.000,
        1.000, 0.333, 1.000,
        1.000, 0.667, 1.000,
        0.167, 0.000, 0.000,
        0.333, 0.000, 0.000,
        0.500, 0.000, 0.000,
        0.667, 0.000, 0.000,
        0.833, 0.000, 0.000,
        1.000, 0.000, 0.000,
        0.000, 0.167, 0.000,
        0.000, 0.333, 0.000,
        0.000, 0.500, 0.000,
        0.000, 0.667, 0.000,
        0.000, 0.833, 0.000,
        0.000, 1.000, 0.000,
        0.000, 0.000, 0.167,
        0.000, 0.000, 0.333,
        0.000, 0.000, 0.500,
        0.000, 0.000, 0.667,
        0.000, 0.000, 0.833,
        0.000, 0.000, 1.000,
        0.000, 0.000, 0.000,
        0.143, 0.143, 0.143,
        0.286, 0.286, 0.286,
        0.429, 0.429, 0.429,
        0.571, 0.571, 0.571,
        0.714, 0.714, 0.714,
        0.857, 0.857, 0.857,
        0.000, 0.447, 0.741,
        0.50, 0.5, 0
    ]
).astype(np.float32)
color_list = color_list.reshape((-1, 3)) * 255

In [2]:
colors = [(color_list[_]).astype(np.uint8) \
                  for _ in range(len(color_list))]

In [11]:
colors = [(color_list[_]).astype(np.uint8) \
                  for _ in range(len(color_list))]*5

In [12]:
len(colors)

400

In [13]:
olors = np.array(colors, dtype=np.uint8).reshape(len(colors), 1, 1, 3)

In [14]:
olors.shape

(400, 1, 1, 3)

In [2]:
0%3

0

In [3]:
a = {}
a['a'] = [1,2]

In [4]:
a


{'a': [1, 2]}